# Identifying Port Calls in the PIERS BoL Data

### Problem Statement:

The PIERS data do not have clean arrival or departure dates; instead, cargo imported from a given shipping vessel at a given port is spread over multiple dates, often spanning several days up to several weeks, even though the vessels spend only a matter of hours at any given port call. The reason for this spread is not precisely known at this time, but given the underlying data come from US Customs, we expect it may be related to the time required to process the individual bills. 

Therefore we need to identify clusters of dates that indicate a unique port call for each vessel, with the goal of assigning the earliest date in the cluster as the actual arrival/departure date for imports/exports. 

Potential uses of this information include analyzing frequency of service at each port, identification of individual voyages, and many others. 

In [6]:
#prelims
import pandas as pd #v2.1.3
import polars as pl #v0.20.7
import plotly_express as px #v0.4.1 
import missingno as msno #v0.5.2
import datetime as dt
from sklearn.cluster import HDBSCAN

#display settings
pd.set_option('display.max_columns', None)

#enable string cache for polars categoricals
pl.enable_string_cache()

In [7]:
# basic import from data_cleaning notebook

#set paths
imports_path = 'data/raw_parquet/imports/'
exports_path = 'data/raw_parquet/exports/'
#init lazy dataframes
imports_lf = (
    pl.scan_parquet(imports_path+'*.parquet', parallel='columns')
    .with_columns([
        #create 2-digit hs code
        pl.col('hs_code').str.slice(0, length=2).alias('hs_2d'),
        #create year and month columns
        pl.col('date_arrival').dt.year().alias('year'),
        pl.col('date_arrival').dt.strftime('%Y%m').alias('month'),
        #convert zero volume values to null
        pl.col('teus').replace(0,None),
        pl.col('weight').replace(0,None),
        pl.col('qty').replace(0,None),
        #create direction column
        pl.lit('import').cast(pl.Categorical).alias('direction'), 
        #create lane_id
        (pl.col('departure_port_code').cast(pl.Utf8)+'_'+pl.col('arrival_port_code').cast(pl.Utf8))
        .cast(pl.Categorical)
        .alias('lane_id')
        ])
    #get lane name 
    .with_columns(
            pl.col('departure_port_name').drop_nulls().mode().first().over('lane_id').alias('best_departure_port_name'),
            pl.col('arrival_port_name').drop_nulls().mode().first().over('lane_id').alias('best_arrival_port_name')
        )
        .with_columns(
            (pl.col('best_departure_port_name').cast(pl.Utf8)+' — '+pl.col('best_arrival_port_name').cast(pl.Utf8))
            .str.to_titlecase()
            .cast(pl.Categorical)
            .alias('lane_name')
        )
        .drop('best_departure_port_name', 'best_arrival_port_name')
    #get most commonly used carrier name for each scac and vise-versa to normalize changes in names and codes 
    .with_columns(
        pl.col('carrier_name').drop_nulls().mode().first().over('carrier_scac')
        .alias('unified_carrier_name')
    )
    .with_columns(
        pl.col('carrier_scac').drop_nulls().mode().first().over('unified_carrier_name')
        .alias('unified_carrier_scac')
    )
    .with_columns(
        #create voyage_id from year, vessel IMO, voyage number, and origin country
        (pl.col('year').cast(pl.Utf8)+'_'+
         pl.col('vessel_id').cast(pl.Utf8)+'_'+
         pl.col('voyage_number').cast(pl.Utf8)+'_'+
         pl.col('origin_territory').cast(pl.Utf8))
        .cast(pl.Categorical).alias('voyage_id')
    )
)
exports_lf = (
    pl.scan_parquet(exports_path+'piers_exports_complete.parquet', parallel='columns')
    .with_columns([
        #create 2-digit hs code
        pl.col('hs_code').str.slice(0, length=2).alias('hs_2d'),
        #create year and month columns
        pl.col('date_departure').dt.year().alias('year'),
        pl.col('date_departure').dt.strftime('%Y%m').alias('month'),
        #convert zero volume values to null
        pl.col('teus').replace(0,None),
        pl.col('weight').replace(0,None),
        pl.col('qty').replace(0,None),
        #create direction column
        pl.lit('export').cast(pl.Categorical).alias('direction'),
        #create lane_id
        (pl.col('departure_port_code').cast(pl.Utf8)+'_'+pl.col('arrival_port_code').cast(pl.Utf8))
        .cast(pl.Categorical)
        .alias('lane_id')
        ])
    #get lane name 
    .with_columns(
            pl.col('departure_port_name').drop_nulls().mode().first().over('lane_id').alias('best_departure_port_name'),
            pl.col('arrival_port_name').drop_nulls().mode().first().over('lane_id').alias('best_arrival_port_name')
        )
        .with_columns(
            (pl.col('best_departure_port_name').cast(pl.Utf8)+' — '+pl.col('best_arrival_port_name').cast(pl.Utf8))
            .str.to_titlecase()
            .cast(pl.Categorical)
            .alias('lane_name')
        )
        .drop('best_departure_port_name', 'best_arrival_port_name')
    #get most commonly used carrier name and scac 
    .with_columns(
        pl.col('carrier_name').drop_nulls().mode().first().over('carrier_scac')
        .alias('unified_carrier_name')
    )
    .with_columns(
        pl.col('carrier_scac').drop_nulls().mode().first().over('unified_carrier_name')
        .alias('unified_carrier_scac')
    )
    .with_columns(
        #create voyage_id from year, vessel IMO, voyage number, and origin country
        (pl.col('year').cast(pl.Utf8)+'_'+
         pl.col('vessel_id').cast(pl.Utf8)+'_'+
         pl.col('voyage_number').cast(pl.Utf8)+'_'+
         pl.col('dest_territory').cast(pl.Utf8))
        .cast(pl.Categorical).alias('voyage_id')
    )
)

In [8]:
# function defs from data_cleaning

#fill nulls in volume cols with mean
def fill_volume(lf):
    '''ad hod function to fill volume columns with their means'''
    return (
        lf
        .with_columns([
            pl.col('teus').replace(0,None).fill_null(strategy='mean'),
            pl.col('weight').replace(0,None).fill_null(strategy='mean'),
            pl.col('qty').replace(0,None).fill_null(strategy='mean')
            ])
        )

#plotly graph inspecting nulls over time by group
def nulls_over_time_plotly(data_lf, group_var, time_var, value_var, title=False):
    '''
    Plots proportion of null values over time by group.
    INPUTS:
        data_lf - polars lazyframe containing the relevant data
        group_var - str - the name of the column by which to group
        time_var - str - the name of the time column (e.g., year, month) over which values will be counted
        value_var - str - the name of the column containing the variable in question
        title (default=False) - str - the title of the graph
    OUTPUT:
        a plotly express figure
    DEPENDS ON:
        polars
        plotly express 
    '''
    df = (
        #select relevant columns
        data_lf.select([group_var, time_var, value_var])
        #group by, creating null count and non-null count cols
        .group_by(group_var, time_var)
        .agg([pl.col(value_var).null_count().alias('null_count'),
                pl.col(value_var).count().alias('count')])
        #compute percent null and fill new column
        .with_columns((pl.col('null_count')/(pl.col('count')+pl.col('null_count'))).alias('null_percent'))
        #cast group col to string to allow sensible ordering of legend
        .cast({group_var:pl.Utf8})
        #sort by date (to allow proper visualization of lines) and group (for legend ordering) 
        .sort(time_var, group_var)
    ).collect()
    #plot
    fig = px.line(
        data_frame=df,
        x=time_var, y='null_percent',
        color=group_var,
        title= 'Count of nulls over time by source frame.' if not title else title
    )
    fig.show()
    del df

#plotly graph inspecting nulls by group
def nulls_by_group_plotly(data_lf, group_var, value_var, title=False):
    '''Plots proportion of null values in the given groups'''
    df = (
        data_lf.select([group_var, value_var])
        .group_by(group_var)
        .agg([pl.col(value_var).null_count().alias('null_count'),
                pl.col(value_var).count().alias('count')])
        .with_columns((pl.col('null_count')/(pl.col('count')+pl.col('null_count'))).alias('null_percent'))
        .cast({group_var:pl.Utf8})
        .sort('null_percent', descending=True)
    ).collect()
    #plot
    fig = px.bar(
        data_frame=df,
        x=group_var, y='null_percent',
        title= 'Null percent by group.' if not title else title
    )
    fig.show()

#fill nulls over groups given a single unique value per group
def fill_nulls_by_group(data_lf, group_vars, val_var):
    '''Fills null values by group if and only if the val_var for that group contains exactly one non-null unique value.
    INPUTS:
        data_lf - polars lazyframe containing the relevant data
        group_vars - iterable - the names of the columns by which groups will be created
        val_var - string - the name of the column in which nulls will be filled
    OUTPUT:
        filled_lf - the resultant lazyframe 
    DEPENDS ON:
        polars - current version written in polars 0.20.1
    '''
    filled_lf = (
        data_lf.with_columns(
            #if the group contains exactly one unique value: 
            pl.when(pl.col(val_var).drop_nulls().unique(maintain_order=True).len().over(group_vars)==1)
            #then fill the group with that value
            .then(pl.col(val_var).fill_null(pl.col(val_var).drop_nulls().unique(maintain_order=True).first().over(group_vars)))
            #otherwise do nothing
            .otherwise(pl.col(val_var))
            )
        )
    return filled_lf

def is_one2one(lf, col1, col2):
    '''checks if the two pl.LazyFrame columns constitute a 1-1 pairing'''
    forward = lf.group_by(col1).agg(pl.col(col2).drop_nulls().n_unique()).select(col2).max().collect().item() == 1
    back = lf.group_by(col2).agg(pl.col(col1).drop_nulls().n_unique()).select(col1).max().collect().item() == 1
    return (forward and back)

def count_unique_by_group(lf, group_vars, val_var):
    '''returns a dataframe of unique observations for the value variable over each group'''
    df = (
        lf.group_by(group_vars)
        .agg(
            pl.col(val_var).unique().alias('unique_values'),
            pl.col(val_var).n_unique().alias('n_unique')
        )
        .drop_nulls()
        .sort('n_unique', descending=True)
        .collect()
    )
    return df

def add_primary_carrier(lf):
    '''alternative ad hoc function to find primary carrier for each vessel and indicate cargo sharing'''
    lf = (
        lf.with_columns(
            pl.col('teus').sum()
            .over('vessel_id', 'month', 'unified_carrier_scac')
            .alias('sum_teus')
            )
        .with_columns(
            pl.col('unified_carrier_scac')
            .sort_by('sum_teus', descending=True)
            .drop_nulls().first()
            .over('vessel_id', 'month')
            .alias('vessel_owner')
            )
        #add bool col if bol is from primary carrier
        .with_columns(
            (pl.col('unified_carrier_scac')==pl.col('vessel_owner'))
            .alias('primary_cargo')
            )
    )
    return lf

def sharing_over_time_plotly(data_lf, group_var, include_missing_voyages=True, limit=10, title=False):
    '''
    Plots proportion of shared cargo over time (months) by group_var.
    INPUTS:
        data_lf - polars lazyframe containing the relevant data
        group_var - str - the name of the column by which to group
        include_missing_voyages - bool - default=True, when False, drops missing voyages
        title (default=False) - str - the title of the graph
    OUTPUT:
        a plotly express figure
    DEPENDS ON:
        polars
        plotly express 
    '''
    if not include_missing_voyages:
        df = data_lf.drop_nulls('voyage_id')
    else:
        df = data_lf
    
    df = (
        #select relevant columns
        df.select([group_var, 'month', 'primary_cargo', 'teus'])
        #sum teus over each group-month-shared 
        .group_by(group_var, 'month')
        .agg(
            (pl.col('teus')*pl.col('primary_cargo')).sum().alias('total_primary'),
            pl.col('teus').sum().alias('total_teus')
        )
        #create proportion shared
        .with_columns((1-(pl.col('total_primary')/pl.col('total_teus'))).alias('prop_shared'))
        #cast group col to string to allow sensible ordering of legend
        .cast({group_var:pl.Utf8})
        #sort by date (to allow proper visualization of lines) and group (for legend ordering) 
        .sort('month')
    ).collect()

    #limit categories
    top_groups = (
        data_lf.group_by(group_var)
        .agg(pl.col('teus').sum())
        .sort('teus', descending=True)
        .select(group_var)
        .limit(limit)
        .collect()
        .to_series()
        .cast(pl.Utf8)
    )
    
    #plot
    fig = px.line(
        data_frame=df.filter(pl.col(group_var).is_in(top_groups)).with_columns(pl.col('month').str.to_datetime('%Y%m')),
        x='month', y='prop_shared',
        color=group_var,
        title= 'Proportion of shared cargo over time.' if not title else title,
        labels={
            'prop_shared':'Proportion of cargo from non-primary carrier',
            'month':'Month'
        }
    )
    fig.show()


In [9]:
# fill volumes and add primary carrier 
imports_lf = fill_volume(imports_lf)
exports_lf = fill_volume(exports_lf)   
#add primary carrier and shared cargo columns to lfs
imports_lf = add_primary_carrier(imports_lf)
exports_lf = add_primary_carrier(exports_lf)


In [10]:
df = (
    imports_lf
    .group_by('date_arrival', 'vessel_id', 'arrival_port_code')
    .agg(
        pl.col('teus').sum().alias('sum_teus'),
        pl.col('bol_id').count().alias('num_bills')
    )
    .with_columns(
        (pl.col('vessel_id').cast(pl.Utf8)+'_'+pl.col('arrival_port_code').cast(pl.Utf8)).cast(pl.Categorical).alias('vessel_port_pair')
    )
    .sort('date_arrival')
    .collect()
)

df.head()

date_arrival,vessel_id,arrival_port_code,sum_teus,num_bills,vessel_port_pair
datetime[μs],i32,cat,f64,u32,cat
2005-01-01 00:00:00,8601410,"""2811""",2.196793,1,"""8601410_2811"""
2005-01-01 00:00:00,9153850,"""3001""",4.393585,2,"""9153850_3001"""
2005-01-01 00:00:00,9036002,"""0005""",2.196793,1,"""9036002_0005"""
2005-01-01 00:00:00,9233208,"""2014""",2.196793,1,"""9233208_2014"""
2005-01-01 00:00:00,8702915,"""1401""",2.196793,1,"""8702915_1401"""


In [11]:
date = dt.datetime(2023,1,1)

toppairs = (
    df
    .filter(pl.col('date_arrival')>date)
    .group_by('vessel_port_pair')
    .agg(pl.col('date_arrival').n_unique().alias('num_dates'))
    .drop_nulls()
    .sort('num_dates', descending=True)
    .limit(20)
    .select('vessel_port_pair')
    .to_series()
)

In [ ]:
for pair in toppairs:
    px.bar(
        df.filter(
            pl.col('vessel_port_pair')==pair,
            pl.col('date_arrival')>date
            ), 
        x='date_arrival', y='sum_teus', color='vessel_port_pair'
    ).show()

see if voyage id is unique-ish per cluster

In [36]:
toppair = toppairs[1]

pair_1d = (
    df.filter(
            pl.col('vessel_port_pair')==toppair,
            pl.col('date_arrival')>date
            )
    .select('date_arrival', pl.col('sum_teus').round())
    .select(pl.exclude('sum_teus').repeat_by('sum_teus').explode())
)

px.histogram(pair_1d, nbins=365)

In [37]:
#instantiate clusterer
clusterer = HDBSCAN(min_cluster_size=15)
#get clusters
clusterer.fit(pair_1d)
#add back to pair_1d
clustered_pair_id = (
    pair_1d.with_columns(
    pl.Series(name='cluster', values=clusterer.labels_)
    )
)

In [38]:
px.histogram(data_frame=clustered_pair_id, x='date_arrival', color='cluster', nbins=365)

In [39]:
clustered_pair_id.group_by('cluster').agg(pl.col('date_arrival').n_unique().sort(descending=True)).head(20)

cluster,date_arrival
i64,u32
27,2
18,3
21,3
15,3
3,2
12,3
0,2
24,2
6,3


In [ ]:
test_df = imports_lf.select('date_arrival').group_by('date_arrival').len().collect().limit(20)

In [ ]:
what = test_df.select(pl.exclude('len').repeat_by('len').explode())

In [ ]:
#ref


data = pd.read_csv('data_set.csv')

clusterer = hdbscan.HDBSCAN(min_cluster_size=4,min_samples=8)
clusterer.fit(data)
data['cluster'] = clusterer.labels_

fig = px.bar(data,x='price',y='quantity',color='cluster',orientation='v')
fig.show()